In [2]:
!pip install rasterio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 58.2 MB/s eta 0:00:00


In [3]:
import os
import rasterio
from rasterio.mask import mask
import fiona
from shapely.geometry import shape

In [ ]:
# Set up variables
geotiff_folder = "/path/to/your/geotiff/folder"
shapefile_path = "/path/to/your/shapefile.shp"
output_folder = "/path/to/your/output/folder/Clipped_Land"


In [ ]:
def clip_rasters_with_shapefile(geotiff_folder, shapefile_path, output_folder):
    # Ensure output directory exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Read in the shapefile using fiona
    with fiona.open(shapefile_path, "r") as shapefile:
        # We'll assume that your shapefile has only one feature (polygon) for clipping
        # If you have multiple features, you'd have to loop through each of them
        geom = [shape(shapefile[0]['geometry'])]

    # Loop through each geotiff in the folder
    for root, dirs, files in os.walk(geotiff_folder):
        for file in files:
            if file.endswith(".tif"):
                geotiff_path = os.path.join(root, file)

                # Open the geotiff using rasterio
                with rasterio.open(geotiff_path, "r") as src:
                    # Clip the geotiff using the shapefile's geometry
                    out_image, out_transform = mask(src, geom, crop=True)
                    out_meta = src.meta.copy()

                    # Update metadata with new dimensions, transform
                    out_meta.update({
                        "driver": "GTiff",
                        "height": out_image.shape[1],
                        "width": out_image.shape[2],
                        "transform": out_transform
                    })

                    # Write the clipped raster to the new folder with the `_clipland` extension
                    with rasterio.open(os.path.join(output_folder, os.path.splitext(file)[0] + "_clipland.tif"), "w", **out_meta) as dest:
                        dest.write(out_image)




In [ ]:
# Example Usage
geotiff_folder = "/path/to/your/geotiff/folder"
shapefile_path = "/path/to/your/shapefile.shp"
output_folder = "/path/to/your/output/folder/Clipped_Land"
clip_rasters_with_shapefile(geotiff_folder, shapefile_path, output_folder)

####Second iteration####


In [ ]:
!pip install rasterio

In [ ]:
import os
import rasterio
from rasterio.mask import mask
import fiona
from shapely.geometry import shape

In [ ]:
def clip_rasters_with_shapefile(geotiff_folder, shapefile_path, output_folder):
    # Ensure output directory exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Read in the shapefile using fiona
    with fiona.open(shapefile_path, "r") as shapefile:
        # Loop through each feature in the shapefile
        for feature in shapefile:
            geom = [shape(feature['geometry'])]

            # Loop through each geotiff in the folder
            for root, dirs, files in os.walk(geotiff_folder):
                for file in files:
                    if file.endswith(".tif"):
                        geotiff_path = os.path.join(root, file)

                        # Open the geotiff using rasterio
                        with rasterio.open(geotiff_path, "r") as src:
                            # Clip the geotiff using the polygon's geometry
                            out_image, out_transform = mask(src, geom, crop=True)
                            out_meta = src.meta.copy()

                            # Update metadata with new dimensions, transform
                            out_meta.update({
                                "driver": "GTiff",
                                "height": out_image.shape[1],
                                "width": out_image.shape[2],
                                "transform": out_transform
                            })

                            # Construct an output filename based on the original filename and the feature's ID
                            output_filename = os.path.splitext(file)[0] + f"_clipland_{feature['id']}.tif"

                            # Write the clipped raster to the new folder
                            with rasterio.open(os.path.join(output_folder, output_filename), "w", **out_meta) as dest:
                                dest.write(out_image)



In [ ]:
# Example Usage
geotiff_folder = "/path/to/your/geotiff/folder"
shapefile_path = "/path/to/your/shapefile.shp"
output_folder = "/path/to/your/output/folder/Clipped_Land"
clip_rasters_with_shapefile(geotiff_folder, shapefile_path, output_folder)
